After some effort , this is my first ML model.

In [ ]:
#import tools (latter maybe news tools will be needed)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


In [ ]:
df = pd.read_csv("../input/insurance/insurance.csv")
df.shape

In [ ]:
#checking all informations about the data set
df.head()

In [ ]:
df.info()

In [ ]:
df["sex"].value_counts()

In [ ]:
#548 different numbers for BMI(body mass index)
len(df["bmi"].value_counts())     

In [ ]:
df["children"].value_counts()

In [ ]:
df["smoker"].value_counts()

In [ ]:
df["region"].value_counts()

In [ ]:
fig, ax =plt.subplots()
ax.hist(df["bmi"])
ax.set(ylabel ="frequency(total number is 1338)",
       xlabel ="BMI(body max index)");

In [ ]:
fig, ax =plt.subplots()
ax.hist(df["charges"]);
ax.set(ylabel ="frequency",
       xlabel ="price(charge) of insurance");

In [ ]:
less_than_10k =[]
between_10k_and_20k =[]
more_than_20k =[]
for x in df["charges"]:
    if x<10001:
        less_than_10k.append(x)
    if 10000<x<20001:
        between_10k_and_20k.append(x)
    if x>20000:
        more_than_20k.append(x)
print(f"{len(less_than_10k)} is number of people paying less than 10 000$ for insurance and \n{len(between_10k_and_20k)} is a number of people paying between 10 000$ and 20 000$ for insurance \n{len(more_than_20k)} is a number people are paying 20 000$ or above for insurance")
    

In [ ]:
df.loc[df["sex"]=="male"]

In [ ]:
#mean (average) price of the health insurance
round(df.charges.mean(),2)

In [ ]:
only_male_data =df.loc[df["sex"]=="male" ]

In [ ]:
#average charges for the health insurance for the males
only_male_data["charges"].mean()

In [ ]:
only_female_data =df.loc[df["sex"]=="female" ]

#average charges for the health insurance for the females
only_female_data["charges"].mean()

In [ ]:
smoker =df.loc[df["smoker"]=="yes" ]
nonsmoker =df.loc[df["smoker"]=="no"]


In [ ]:
print(f"averege smoker cost of the health insurance is {round(smoker.charges.mean(),2)}$, while a nosmoker pays {round(nonsmoker.charges.mean(),2)}$ which is {round(smoker.charges.mean()/ nonsmoker.charges.mean(),2) } times more expensive")

In [ ]:
smoker["charges"].mean()

In [ ]:
BMI_less_than_average =df.loc[df["bmi"]<30.66 ]
BMI_abov_than_average =df.loc[df["bmi"]>30.66 ]

In [ ]:
print(f" people with the BMI less than average pays average about {round(BMI_less_than_average.charges.mean(),2)}$ for the health insurance, while people with the BMI above the average pays {round(BMI_abov_than_average.charges.mean(),2)}$")

In [ ]:
round(BMI_abov_than_average.charges.mean(),2)

In [ ]:
fig, ax =plt.subplots()
ax.hist(df["age"])
ax.set(ylabel ="frequency",
       xlabel ="age(the lowest age group is 18yrs)");

In [ ]:
#peparing a data for transforming - strings like male/female to numbers like 0,1  so our model can undestand them
#first we need to import a tools
from sklearn.preprocessing import OneHotEncoder
ohe =OneHotEncoder(sparse =False)


In [ ]:
#test and example
ohe.fit_transform(df[["sex"]])

In [ ]:
#spliting the data to y variable(charges /price of the insurance) and X variable(everything else) 
X=df.drop("charges", axis=1)
y =df["charges"]
y

In [ ]:
df["children"].value_counts()

In [ ]:
#transforming all of the objects to numerical data via column transformer
from sklearn.compose import make_column_transformer
column_trans = make_column_transformer(
    (OneHotEncoder(), ["sex", "smoker", "region"]), remainder ="passthrough")

In [ ]:
transX =column_trans.fit_transform(X)

In [ ]:
#raw data
pd.DataFrame(transX)

In [ ]:
dfcolumns =[ 'sex female',"sex male",'smoker no', "smoker yes" ,'region north east', "region northwest", "region southeast", "region southwest", "age" ,"BMI", "num of childern"]

In [ ]:
# testing model with RandomForestRegressor
np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor
regresor =RandomForestRegressor()

In [ ]:
np.random.seed(42)
X_train,X_test, y_train, y_test = train_test_split(transX, y , test_size=0.2)
regresor.fit(X_train, y_train)
regresor.score(X_test, y_test)

In [ ]:
#lets try RidgeRegression Model
np.random.seed(42)
from sklearn.linear_model import Ridge
ridge =Ridge()
ridge.fit(X_train,y_train)
ridge.score(X_test, y_test)

In [ ]:
#value of the price that our RandomForestRegressor model has predicted
y_preds = regresor.predict(X_test)
y_preds[:10]

In [ ]:
np.array(y_test[:10])

In [ ]:
df.iloc[3]

In [ ]:
#compare predictions from the truth (y_test is true value, y_preds is predicted value by my model)

from sklearn.metrics import mean_absolute_error
print(f" on average charges for health insurance that model predict is about {round(mean_absolute_error(y_test, y_preds),2)} $ from real value that was charged")

In [ ]:
ytst_yprds = pd.DataFrame(data={"acctual values ": y_test, "predicted values":y_preds})
ytst_yprds

In [ ]:
ytst_yprds["difference in $"] =round(ytst_yprds["acctual values "] - ytst_yprds["predicted values"],2)

In [ ]:
ytst_yprds.head()

In [ ]:
ytst_yprds.loc[ytst_yprds["difference in $"]>3000]

In [ ]:
#time to introduce cross validation score - that splits our model to the 5 smaller models
cvr2 =cross_val_score(regresor, transX, y, cv=5, scoring=None)
cvr2

In [ ]:
cvr2.mean()

In [ ]:
np.random.seed(42)
regresor.score(X_test, y_test)

In [ ]:
#using cross validation score to calculate mean absolute error
np.random.seed(42)
cv_mae =cross_val_score(regresor, transX, y , cv =5, scoring ="neg_mean_absolute_error")
cv_mae, cv_mae.mean()

In [ ]:
mean_absolute_error(y_test, y_preds)

In [ ]:
#parameters that are important to make deicision in our model
regresor.get_params()

In [ ]:
%time
#searching for the best parameters so we can improve model
rf_grid = {"n_estimators": np.arange(10,100,10) , 
           "min_samples_leaf": np.arange(1,20,2), "max_depth" :[None, 3,5,10], "min_samples_split": np.arange(2,20,2),
           "max_features" : [0.5, 1, "sqrt", "auto"]}

rand_search = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1 ,random_state=42),
                                param_distributions= rf_grid, n_iter=100, cv=5, verbose=True)

rand_search.fit(X_train,y_train)

In [ ]:
#best parameters
rand_search.best_params_

In [ ]:
#testing score with new better parameters and getting a new slightly improved model
np.random.seed(42)
rand_search.score(X_test, y_test)

In [ ]:
#making the new model with a new improved parameters 
regresor_mod =RandomForestRegressor(n_estimators=20, min_samples_leaf=9, min_samples_split=6, max_features="auto", max_depth=5)

In [ ]:
np.random.seed(42)
regresor_mod.fit(X_train,y_train)
regresor_mod.score(X_test, y_test)

In [ ]:
#compare with the first model
np.random.seed(42)
regresor.score(X_test, y_test)

In [ ]:
#features that had strongest impact on our model
regresor.feature_importances_

In [ ]:
regresor_mod.feature_importances_

In [ ]:
dfcolumns

In [ ]:
len(regresor_mod.feature_importances_)

In [ ]:
feat_data = zip(dfcolumns,regresor_mod.feature_importances_  )

In [ ]:
a =dict(feat_data)

In [ ]:
a

In [ ]:
dataf =pd.DataFrame(a, index=[0])

In [ ]:
# this  dataframe is showing us which features had most impact on our model. (for example: user is a smoker or a non-smoker; then body mass index ; then age)
dataf

In [ ]:
#visual presentation of the most important features 
fix, ax =plt.subplots()
ax.barh(dataf.columns, regresor_mod.feature_importances_);